# Complete Geocoding Demo

Creates source and destination tables, and a process to geocode addresses that haven't been processed yet.

## Setup Instructions

1. Go to https://account.here.com/ and create a free account
2. Once logged in, go to REST APIs > Access Manager
3. Click "Create new app" > give it a name > click "Create app"
4. Click "Create API key"
5. Copy the API key and replace 'YOUR_API_KEY_HERE' in the SECRET creation below

## Create DB and Schema

In [ ]:
USE ROLE ACCOUNTADMIN;
CREATE DATABASE IF NOT EXISTS DEMO_GEOCODE;
CREATE SCHEMA IF NOT EXISTS DEMO_GEOCODE.ADDRESS_PROCESSING;

USE DATABASE DEMO_GEOCODE;
USE SCHEMA ADDRESS_PROCESSING;

## Create Network Rule for HERE API

In [ ]:
CREATE OR REPLACE NETWORK RULE here_geocode_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('geocode.search.hereapi.com');

## Create Secret to Store API Key

**IMPORTANT:** Uncomment the SECRET_STRING line and replace 'YOUR_API_KEY_HERE' with your actual HERE API key

In [ ]:
CREATE OR REPLACE SECRET here_api_key_secret
TYPE = GENERIC_STRING;
-- SECRET_STRING = 'YOUR_API_KEY_HERE';

## Create External Access Integration

In [ ]:
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION here_geocode_integration
  ALLOWED_NETWORK_RULES = (here_geocode_network_rule)
  ALLOWED_AUTHENTICATION_SECRETS = (DEMO_GEOCODE.ADDRESS_PROCESSING.here_api_key_secret)
  ENABLED = true;

## Create Python UDF to Call HERE API

In [ ]:
CREATE OR REPLACE FUNCTION geocode_address(address_string STRING)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.10
HANDLER = 'geocode_address'
EXTERNAL_ACCESS_INTEGRATIONS = (here_geocode_integration)
PACKAGES = ('snowflake-snowpark-python','requests')
SECRETS = ('here_api_key' = here_api_key_secret)
AS
$$
import _snowflake
import requests
import json
import urllib.parse

session = requests.Session()

def geocode_address(address_string):
    try:
        api_key = _snowflake.get_generic_secret_string('here_api_key')
        encoded_address = urllib.parse.quote(address_string)
        url = f"https://geocode.search.hereapi.com/v1/geocode?q={encoded_address}&apiKey={api_key}"
        response = session.get(url)
        return response.text
    except Exception as e:
        return json.dumps({"error": str(e)})
$$;

## Create Source_Addresses Table

In [ ]:
CREATE OR REPLACE TABLE Source_Addresses (
    Name VARCHAR(255),
    Department VARCHAR(100),
    Address_ID INTEGER AUTOINCREMENT,
    Address_Source_ID VARCHAR(100),
    Address VARCHAR(500),
    GeoCoded VARCHAR(3) DEFAULT 'No'
);

## Create Geocoded_Addresses Table

In [ ]:
CREATE OR REPLACE TABLE Geocoded_Addresses (
    Geocoded_ID INTEGER AUTOINCREMENT,
    Name VARCHAR(255),
    Department VARCHAR(100),
    Address_ID INTEGER,
    Address_Source_ID VARCHAR(100),
    Address VARCHAR(500),
    Street VARCHAR(255),
    City VARCHAR(100),
    State VARCHAR(100),
    Zip VARCHAR(20),
    Lat FLOAT,
    Long FLOAT,
    Geocoded_Timestamp TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP() 
);

## Insert Sample Addresses

In [ ]:
INSERT INTO Source_Addresses (Name, Address_Source_ID, Department, Address, GeoCoded) VALUES
    ('Miami Beach Property', 'A1', 'Sales', '4601 Collins Ave Miami Beach FL 33140', 'No'),
    ('Rochester Home', 'A2','Marketing', '114 Orland Rd Rochester NY 14622', 'No'),
    ('Greenville Residence','A3', 'Sales', '103 Autumn Rd Greenville SC 29650', 'No'),
    ('Clemson1', 'A4', 'Student', '217 W Main ST Central SC 29630', 'No'),
    ('Clemson2', 'A5', 'Student', '119 N Townville St Seneca SC 29678', 'No'),
    ('Clemson3', 'A6', 'Student', '356 Clemosn St Clemson SC 29631', 'No');

## Create Main Processing Procedure

In [ ]:
CREATE OR REPLACE PROCEDURE Process_Ungeocoded_Addresses()
RETURNS STRING
LANGUAGE SQL
AS
$$
DECLARE
    processed_count INTEGER DEFAULT 0;
    error_count INTEGER DEFAULT 0;
    total_count INTEGER DEFAULT 0;
    result_message STRING;
BEGIN
    SELECT COUNT(*) INTO :total_count
    FROM Source_Addresses
    WHERE GeoCoded = 'No';
    
    INSERT INTO Geocoded_Addresses (Name, Department, Address_ID, Address_Source_ID, Address, Street, City, State, Zip, Lat, Long)
    WITH addresses_to_process AS (
        SELECT Name, Department, Address_ID, Address_Source_ID, Address
        FROM Source_Addresses
        WHERE GeoCoded = 'No'
    ),
    geocoded_data AS (
        SELECT a.Name, a.Department, a.Address_ID, a.Address_Source_ID, a.Address,
               PARSE_JSON(geocode_address(a.Address))::VARIANT AS api_response
        FROM addresses_to_process a
    ),
    parsed_results AS (
        SELECT g.Name, g.Department, g.Address_ID, g.Address_Source_ID, g.Address,
               f.value:address.street::STRING AS street,
               f.value:address.city::STRING AS city,
               f.value:address.state::STRING AS state,
               f.value:address.postalCode::STRING AS zip,
               f.value:position.lat::FLOAT AS lat,
               f.value:position.lng::FLOAT AS long
        FROM geocoded_data g, LATERAL FLATTEN(input => g.api_response:items) f
    )
    SELECT Name, Department, Address_ID, Address_Source_ID, Address, street, city, state, zip, lat, long
    FROM parsed_results WHERE lat IS NOT NULL;
    
    processed_count := SQLROWCOUNT;
    
    UPDATE Source_Addresses SET GeoCoded = 'Yes'
    WHERE Address_ID IN (
        SELECT Address_ID FROM Geocoded_Addresses 
        WHERE Address_ID NOT IN (SELECT Address_ID FROM Source_Addresses WHERE GeoCoded = 'Yes')
    ) AND GeoCoded = 'No';
    
    error_count := total_count - processed_count;
    result_message := 'Processing Complete. Total: ' || total_count || ', Success: ' || processed_count || ', Errors: ' || error_count;
    RETURN result_message;
END;
$$;

## View Addresses That Need Geocoding

In [ ]:
SELECT Address_ID, Address_Source_ID, Name, Department, Address, GeoCoded
FROM Source_Addresses
WHERE GeoCoded = 'No'
ORDER BY Address_ID;

## Execute the Geocoding Process

In [ ]:
CALL Process_Ungeocoded_Addresses();

## View All Source Addresses

In [ ]:
SELECT Address_ID, Address_Source_ID, Name, Department, Address, GeoCoded
FROM Source_Addresses
ORDER BY Address_ID;

## View All Geocoded Results

In [ ]:
SELECT Geocoded_ID, Name, Department, Address_ID, Address_Source_ID, Address,
       Street, City, State, Zip, Lat, Long, Geocoded_Timestamp
FROM Geocoded_Addresses
ORDER BY Address_ID;

## Test with Google Maps Links

In [ ]:
SELECT address, 'https://www.google.com/maps?q=' || lat || ',' || long AS google_maps_link
FROM Geocoded_Addresses WHERE GEOCODED_TIMESTAMP IS NOT NULL LIMIT 5;

## View Joined Data (Source + Geocoded)

In [ ]:
SELECT s.Address_ID, s.Address_Source_ID, s.Name, s.Department,
       s.Address AS Original_Address, s.GeoCoded,
       g.Street, g.City, g.State, g.Zip, g.Lat, g.Long, g.Geocoded_Timestamp
FROM Source_Addresses s
LEFT JOIN Geocoded_Addresses g ON s.Address_ID = g.Address_ID
ORDER BY s.Address_ID;

## Find Failed Addresses

In [ ]:
SELECT s.Address_ID, s.Address_Source_ID, s.Name, s.Department, s.Address, s.GeoCoded
FROM Source_Addresses s
LEFT JOIN Geocoded_Addresses g ON s.Address_ID = g.Address_ID
WHERE g.Address_ID IS NULL AND s.GeoCoded = 'No'
ORDER BY s.Address_ID;

## Create Monitoring Views

In [ ]:
CREATE OR REPLACE VIEW Geocoding_Status_View AS
SELECT 
    COUNT(*) as total_addresses,
    SUM(CASE WHEN GeoCoded = 'Yes' THEN 1 ELSE 0 END) as geocoded_count,
    SUM(CASE WHEN GeoCoded = 'No' THEN 1 ELSE 0 END) as pending_count,
    ROUND(100.0 * SUM(CASE WHEN GeoCoded = 'Yes' THEN 1 ELSE 0 END) / NULLIF(COUNT(*), 0), 2) as success_rate_pct
FROM Source_Addresses;

In [ ]:
SELECT * FROM Geocoding_Status_View;

## Create Batch Processing Procedure

In [ ]:
CREATE OR REPLACE PROCEDURE Process_Ungeocoded_Addresses_Batch(batch_size INTEGER)
RETURNS STRING
LANGUAGE SQL
AS
$$
DECLARE
    processed_count INTEGER DEFAULT 0;
    total_to_process INTEGER DEFAULT 0;
    result_message STRING;
BEGIN
    SELECT COUNT(*) INTO :total_to_process
    FROM (SELECT Address_ID FROM Source_Addresses WHERE GeoCoded = 'No' LIMIT :batch_size);
    
    INSERT INTO Geocoded_Addresses (Name, Department, Address_ID, Address_Source_ID, Address, Street, City, State, Zip, Lat, Long)
    WITH addresses_to_process AS (
        SELECT Name, Department, Address_ID, Address_Source_ID, Address
        FROM Source_Addresses WHERE GeoCoded = 'No' LIMIT :batch_size
    ),
    geocoded_data AS (
        SELECT a.Name, a.Department, a.Address_ID, a.Address_Source_ID, a.Address,
               PARSE_JSON(geocode_address(a.Address))::VARIANT AS api_response
        FROM addresses_to_process a
    ),
    parsed_results AS (
        SELECT g.Name, g.Department, g.Address_ID, g.Address_Source_ID, g.Address,
               f.value:address.street::STRING AS street,
               f.value:address.city::STRING AS city,
               f.value:address.state::STRING AS state,
               f.value:address.postalCode::STRING AS zip,
               f.value:position.lat::FLOAT AS lat,
               f.value:position.lng::FLOAT AS long
        FROM geocoded_data g, LATERAL FLATTEN(input => g.api_response:items) f
    )
    SELECT Name, Department, Address_ID, Address_Source_ID, Address, street, city, state, zip, lat, long
    FROM parsed_results WHERE lat IS NOT NULL;
    
    processed_count := SQLROWCOUNT;
    
    UPDATE Source_Addresses SET GeoCoded = 'Yes'
    WHERE Address_ID IN (
        SELECT Address_ID FROM Geocoded_Addresses 
        WHERE Address_ID NOT IN (SELECT Address_ID FROM Source_Addresses WHERE GeoCoded = 'Yes')
    ) AND GeoCoded = 'No';
    
    result_message := 'Batch Complete. Limit: ' || batch_size || ', Found: ' || total_to_process || ', Success: ' || processed_count;
    RETURN result_message;
END;
$$;

## Process in Batches

In [ ]:
CALL Process_Ungeocoded_Addresses_Batch(100);

## Create Analytics Views

In [ ]:
CREATE OR REPLACE VIEW Geocoding_Analytics_View AS
SELECT DATE_TRUNC('day', Geocoded_Timestamp) as geocoded_date, Department,
       COUNT(*) as addresses_geocoded, 
       COUNT(DISTINCT State) as states_covered,
       COUNT(DISTINCT City) as cities_covered,
       AVG(Lat) as avg_latitude, AVG(Long) as avg_longitude
FROM Geocoded_Addresses
GROUP BY DATE_TRUNC('day', Geocoded_Timestamp), Department
ORDER BY geocoded_date DESC, Department;

In [ ]:
SELECT * FROM Geocoding_Analytics_View;

In [ ]:
CREATE OR REPLACE VIEW Unprocessed_Addresses_View AS
SELECT s.Address_ID, s.Address_Source_ID, s.Name, s.Department, s.Address, s.GeoCoded
FROM Source_Addresses s
LEFT JOIN Geocoded_Addresses g ON s.Address_ID = g.Address_ID
WHERE g.Address_ID IS NULL AND s.GeoCoded = 'No'
ORDER BY s.Address_ID;

In [ ]:
SELECT * FROM Unprocessed_Addresses_View;

## Department-Specific Queries

In [ ]:
-- View by department
SELECT Department, COUNT(*) as total_addresses,
       SUM(CASE WHEN GeoCoded = 'Yes' THEN 1 ELSE 0 END) as geocoded,
       SUM(CASE WHEN GeoCoded = 'No' THEN 1 ELSE 0 END) as pending
FROM Source_Addresses GROUP BY Department ORDER BY Department;

In [ ]:
-- View geocoded results for specific department
SELECT Name, Address, City, State, Zip, Lat, Long, Geocoded_Timestamp
FROM Geocoded_Addresses WHERE Department = 'Sales'
ORDER BY Geocoded_Timestamp DESC;

In [ ]:
-- Summary statistics by department
SELECT Department, COUNT(*) as total_geocoded,
       COUNT(DISTINCT State) as states_covered,
       COUNT(DISTINCT City) as cities_covered,
       MIN(Geocoded_Timestamp) as first_geocoded,
       MAX(Geocoded_Timestamp) as last_geocoded
FROM Geocoded_Addresses
GROUP BY Department ORDER BY total_geocoded DESC;

## Department Summary View

In [ ]:
CREATE OR REPLACE VIEW Department_Summary_View AS
SELECT s.Department,
       COUNT(DISTINCT s.Address_ID) as total_addresses,
       SUM(CASE WHEN s.GeoCoded = 'Yes' THEN 1 ELSE 0 END) as geocoded_count,
       SUM(CASE WHEN s.GeoCoded = 'No' THEN 1 ELSE 0 END) as pending_count,
       COUNT(DISTINCT g.State) as states_covered,
       COUNT(DISTINCT g.City) as cities_covered,
       MIN(g.Geocoded_Timestamp) as first_address_geocoded,
       MAX(g.Geocoded_Timestamp) as most_recent_geocoded
FROM Source_Addresses s
LEFT JOIN Geocoded_Addresses g ON s.Address_ID = g.Address_ID
GROUP BY s.Department ORDER BY total_addresses DESC;

In [ ]:
SELECT * FROM Department_Summary_View;